# Inverse Kinematics and PID control
In this homework assignemnt, you will be tasked with creating an IK solver and controlling a robot arm using PID control (specifically the Franka Panda 7DoF robot). We will be creating two versions of the controller, one that uses pure velocity control and another that using torque control with feedforward dynamics. 

### Import mujoco and numpy

In [51]:
import mujoco
import mujoco_viewer
import numpy as np
import scipy as sp

from autograder import autograder
from sim import Sim

franka_velocity_ctrl_sim = Sim(fname='./franka_emika_panda/scene_velocity_actuators.xml')
franka_torque_ctrl_sim = Sim(fname='./franka_emika_panda/scene_torque_actuators.xml')


### Robot Info: Franka Emika Panda Robot
We will be importing the Franka Panda robot. Take a look at the robot tree in `franka_emika_panda/panda.xml` file. Rather than importing the robot, we will instead import a specific `scene_X.xml` file that specifies different joint actuation on the robot. Note that the `panda.xml` only specifies the robot body and joints. There are two other files that specify the type of actuation on the robot (`vel_act.xml` and `torque_act.xml`) which we will use to demonstrate the different levels of robot control. 

### Assignment Questions 
Go through this notebook and answer all the questions below. There are autograder helpers which will tell you whether you have good code or not. Please use those to ensure that you can safely move on the next problem. This will prevent any bugs from being carried over to the next problem.

## Rotation matrices and error signal
Before we begin, you need to write up a few functions that will help you compute errors on rotation matrices that you will use to control the robot. First, we will write two helper functions `hat` and `unhat` which converts vectors in $\mathbb{R}^3 \to \text{se}(3)$ and back. This will be used when you compute the distance between two rotation matrices. Specifically, the error between two rotation matrices can be defined using the axis-angle formulation when we discussed exponential coordinates. Recall that $R = e^{\hat{\omega}\theta}$ which is a rotation in some axis $\omega$ by the amount of $\theta$. We can convert $R$ into its components $\hat{\omega} \theta \in \mathbb{R}^3$ by using the matrix logarithm $\text{logm}$. We will then need to convert the resulting matrix back into $\mathbb{R}^3$ using the `unhat` operator. We will be using the scipy library to compute `unhat(sp.linalg.logm(R)) = w` where here `w` is combined with $\theta$. To compute the error between two rotation matrices, we simply multiply the two $R_1 R_2^\top$. For example, if $R_1 = R_{WD}$ and $R_2 = R_{WEE}$, then we want an $R$ such that $R_{WD} =R R_{WEE}$. Therefore, $R = R_{WD} R_{WEE}^\top$ is our relative rotation transform.


### Q1 a. Hat, Unhat 
Write a function `hat` that converts a vector $w\in\mathbb{R}^3 \to \text{se}(3)$ and another function `unhat` that goes the opposite direction.

In [52]:
def hat(w):
    what = np.array([[0,-1*w[2],w[1]],
              [w[2],0,-1*w[0]],
              [-1*w[1],w[0],0]])
    return what 

def unhat(R):
    w = np.array([R[2][1],R[0][2],R[1][0]])
                    
    return w

#arr = np.array([1,2,3])
#print(arr)
#print(hat(arr))
#print(unhat(hat(arr)))

In [53]:
autograder.testQ1a(hat, unhat)

Passed Debug test


### Q1 b. Rotation error
Write a function that computes the error between two rotation matrices $R_1, R_2$.

In [61]:
def rot_err(R1, R2):
    #Rd = sp.spatial.transform.Rotation.from_matrix(np.matmul(R1,R2.transpose()))
    R = unhat(sp.linalg.logm(np.matmul(R1,R2.transpose())))
    #print (R)
    #print (Rd.as_rotvec())
    return R




In [62]:
autograder.testQ1b(rot_err)

Passed Debug test


## Inverse Kinematics Solver
We will now create the IK solver. We will use some helper functions to get the rotation, position, and jacobian of a site in the `.xml` file. Look through the files and find the tags for `site`. Specifically, look for the two tags `ee_site` and `target_site` within the `worldbody` tags. These will be the two target frames on a the robot and in the world respectively. Understand the functions below and answer the subsequent questions. 

In [63]:
def getSitePose(model, data, site_name):
    """
        This function takes in the robot model, data, and the site name of the reference frame. 
        The output of the function returns a tuple of a rotation matric R and a position p 
        relative to the world frame. 
    """
    # mujoco specific function
    site_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_SITE, site_name)
    # data structure for obtaining the position and orientation of a site reference frame
    # note that rotation matrices are formated as a 9-dim vector and need to be reshaped
    p = data.site_xpos[site_id]
    R = data.site_xmat[site_id].reshape((3,3))
    return (R, p)

def getSiteJacobian(model, data, site_name):
    """
        This function takes in the robot model, data, and the site name of the reference frame. 
        The output is the total jacobian matrix. Note that mujoco gives jacobian 
        position and rotation matrices which need to be stacked together.
    """
    body_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_SITE, site_name)
    jacp = np.zeros((3, model.nv))
    jacr = np.zeros((3, model.nv))
    mujoco.mj_jacSite(model, data, jacp, jacr, body_id)
    return np.concatenate([jacp, jacr], axis=0)


## Q2. Write the IK step. 
Write function to compute $\Delta q$, for $q_{k+1} = q_k + \Delta q$, where $\Delta q$ is the solution to the minimum joint deviation based on the Taylor expansion of $f(q) - x_\text{des}$, and $f(q)$ is your forward kinematics (assume that this is given as x_\text{curr}).

In [76]:
def calcIKStep(jac, x_des, x_curr):
    """
        This function takes in the jacobian matrix of the x_\text{curr} = f(q) forward kinematics, 
        and the desired frame rotation matrix and position. 
        Assumes x_des, x_curr = (R, p)
        Output of this function computes $\Delta q$ using the jacobian.
    """
    #define an error function, Rcurr and Rdes
    R_curr, p_curr = x_curr
    R_des,  p_des  = x_des
    #calculate rotation and position error
    err = rot_err(R_des, R_curr)
    p_delta =p_des - p_curr
    
    #combine rot and pos error
    error = np.concatenate((p_delta,err))
    #invert jacobian
    jac_inv = np.linalg.pinv(jac)
    
    Step = np.matmul( jac_inv, error)
    #print (Step)
    return Step

def IKSolve(q, jac, x_des, x_curr):
    dq = calcIKStep(jac, x_des, x_curr)
    return q + dq


# Control!
We will now implement different control strategies to move our robot. We will use the function below to simulate the control method used. Make sure you run the cell and load the function. Note that for the following examples you need to calculate the error from the poses of the reference frames.

In [77]:
def simulateCtrl(sim, ctrl_method, render=True, t_max=1000):
    sim.data.qpos = sim.model.key_qpos
    mujoco.mj_step(sim.model, sim.data)
    log = []
    if render:
        viewer = mujoco_viewer.MujocoViewer(sim.model, sim.data)
        viewer.vopt.frame = 3
    for t in range(t_max):
        u = ctrl_method(sim.model, sim.data)
        sim.data.ctrl = u 
        mujoco.mj_step(sim.model, sim.data)
        if render:
            if viewer.is_alive:
                viewer.render()
    if render:
        viewer.close()

## Q3 Velocity control
Create a controller that returns $\Delta q$ given the end-effector pose and the target site pose. Using the `calcIKStep` to compute $\Delta q$. Run the `simulateCtrl` using the `franka_velocity_ctrl_sim` simulation. Describe what you see. 

## Response:
Using this control method, the robot arm tries to reach the target location, but because gravity is not accounted for in this control method, it sags and fails to reach the target location

In [78]:
def velocity_ctrl(model, data):
    """
        Output will be a 7 dim vec that is applied to robot velocity control
        output will be calcIKstep(jac, x_des, x_curr)
    """
    x_des = getSitePose(model, data, "target_site")
    #x_des = getSitePose(model, data, "ee_site")
    x_curr = getSitePose(model, data, "ee_site")   
    jac = getSiteJacobian(model, data, "ee_site")
    
    return calcIKStep(jac, x_des, x_curr) 

In [80]:
simulateCtrl(franka_velocity_ctrl_sim, velocity_ctrl)

Pressed ESC
Quitting.


## Q4 Torque control
Create a  basic PD controller that calculates torque values using $q$ from the `IKSolve` given the end-effector pose and the target site pose. Tune the PD values as best as you can.Describe what you see. 

## Response:

the performance with torque control is better than velocity, but gravity is still unnacounted for, and some sag remains

In [81]:
def torque_ctrl(model, data):
    """
        Output will be a 7 dim vec tau = kp (qdes - qpos) + kd * (-qvel)
        where data.qpos = qpos and data.qvel = qvel are from the robot 
    """
    #1. obtain desired position and EE position
    x_des = getSitePose(model, data, "target_site")
    x_curr = getSitePose(model, data, "ee_site")   
    
    #2. compute Jacobian
    jac = getSiteJacobian(model, data, "ee_site")
    
    #3. solve for change in joint position
        # qk+1 = qk - Jinv(qk)(pdes-p)    
    qdes = IKSolve(data.qpos,jac,x_des,x_curr)
    qvdes = np.zeros(7)
    #print(qvdes)
    
    #No integration neccesary
    #4. set qdes = qk+1, q. des, compute q.. des = -kp(q - q des) - kv( q. - q. des) - ki integral(q - qdes) dt
    kp = 250 #position constant should be 200-300
    kv = 55 #velocity constant 40-65
    #ki = 1 #integral constant
    tau= -kp*(data.qpos - qdes) -kv*(data.qvel-qvdes)
    
    #5. calculate dynamic terms at current joint state q, q.
    
    #6. compute joint torques
        # tau = M(q)q.. des + C(q, q.)q. + V(q)
        
    
    #7. apply to robot
    
    return tau 

In [82]:
simulateCtrl(franka_torque_ctrl_sim, torque_ctrl)

Pressed ESC
Quitting.


## Q4 Dynamic Comp. Torque control
Create a feed forward dynamic compensation controller that calculate target $q$ from the `IKSolve` given the end-effector pose and the target site pose. Tune the PD values as best as you can. Using the provided functions for obtaining the mass matrix and dynamic bias terms. Describe what you see. 

## Response:

once we incorporate the M and B terms, the robot is able to account for gravity (and other bias effects) and reach the target site. 

In [108]:
def getMassMatrix(model, data):
    mass_matrix = np.zeros((model.nv, model.nv))
    mujoco.mj_fullM(model, mass_matrix, data.qM)
    return mass_matrix

def getBiasTerms(model, data):
    return data.qfrc_bias

def feedforward_torque_ctrl(model, data):
    """
        Output will be a 7 dim vec for torque of the form 
        tau = M qddot_des + b
        where qddot_des = kp (qdes - qpos) + kd * (-qvel)
        where data.qpos = qpos and data.qvel = qvel are from the robot 
    """
    #1. obtain desired position and EE position
    x_des = getSitePose(model, data, "target_site")
    x_curr = getSitePose(model, data, "ee_site")
    
    #2. compute Jacobian
    jac = getSiteJacobian(model, data, "ee_site")
    
    #3. solve for change in joint position
        # qk+1 = qk - Jinv(qk)(pdes-p)    
    qdes = IKSolve(data.qpos,jac,x_des,x_curr)
    qvdes = np.zeros(7)
    
    #No integration neccesary
    #4. set qdes = qk+1, q. des, compute q.. des = -kp(q - q des) - kv( q. - q. des) - ki integral(q - qdes) dt
    kp = 250 #position constant should be 200-300
    kv = 55 #velocity constant 40-65
    #ki = 1 #integral constant
    qddot_des= -kp*(data.qpos - qdes) -kv*(data.qvel-qvdes)
    #print(qddot_des)
    
    #5. calculate dynamic terms at current joint state q, q.
    M = getMassMatrix(model, data)
    #print("M")
    #print(M)
    B = getBiasTerms(model, data)
    #print("B")
    #print(B)
    #6. compute joint torques
        # tau = M(q)q.. des + C(q, q.)q. + V(q)
    tau = np.matmul(M,qddot_des) + B
    
    #7. apply to robot
    #print("Tau")
    #print (tau)
    
    return tau 

In [109]:
simulateCtrl(franka_torque_ctrl_sim, feedforward_torque_ctrl)

[ -99.49135623  -43.51816918  -79.63591545  -69.09878808   -0.3974044
   27.15857511 -180.17310309]
M
[[ 1.50865254e+00 -3.61426911e-02  1.37131507e+00  5.09243427e-04
   9.51639626e-02  2.98947839e-04 -8.49094264e-03]
 [-3.61426911e-02  2.79452677e+00 -3.53737552e-02 -1.26786438e+00
  -2.83948065e-02 -9.43958213e-02  3.57179342e-04]
 [ 1.37131507e+00 -3.53737552e-02  1.47131507e+00  5.09243427e-04
   9.51639626e-02  2.98947839e-04 -8.49094264e-03]
 [ 5.09243427e-04 -1.26786438e+00  5.09243427e-04  1.05804436e+00
   2.27639164e-02  1.29921866e-01 -2.36806200e-04]
 [ 9.51639626e-02 -2.83948065e-02  9.51639626e-02  2.27639164e-02
   1.44164678e-01 -1.28063032e-03 -1.57039620e-03]
 [ 2.98947839e-04 -9.43958213e-02  2.98947839e-04  1.29921866e-01
  -1.28063032e-03  1.53980606e-01 -2.05378743e-04]
 [-8.49094264e-03  3.57179342e-04 -8.49094264e-03 -2.36806200e-04
  -1.57039620e-03 -2.05378743e-04  1.06684152e-01]]
B
[-3.48629831e-02 -2.93496866e+01 -3.48708497e-02  2.20422173e+01
  6.6882794